# Coreference resolution in Spacy
The neural coreference resolution module can be combined with spacy.

In [ ]:
# First install compatible version of spacy and neuralcoref
! pip install spacy==2.1.0 neuralcoref

# then restart the kernel

In [ ]:
# download the small spacy model for English if not done already
! python -mspacy download en_core_web_sm

In [1]:
import spacy
from spacy import displacy 

nlp = spacy.load('en_core_web_sm')

In [2]:
doc = nlp('My sister has a dog. She loves him. But he does not love her.')
displacy.render(doc, style='dep',jupyter=True)

Installing neuralcoref package and adding coreference resolution to spacy pipeline

In [3]:
import neuralcoref

nlp_with_coref = spacy.load('en_core_web_sm') # displacy does not work with docs parsed with neuralcoref component
# Add neural coref to SpaCy's pipeline
neuralcoref.add_to_pipe(nlp_with_coref)
# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.

In [4]:
# a naive way to apply the result of coreference resolution
# just reparse the resolved text
doc_resolved = nlp_with_coref('My sister has a dog. She loves him. But he does not love her.')
print(doc_resolved._.coref_clusters)
print(doc_resolved._.coref_resolved)

[My sister: [My sister, She, her], a dog: [a dog, him, he]]
My sister has a dog. My sister loves a dog. But a dog does not love My sister.


In [12]:
doc_resolved_reparsed = nlp(doc_resolved._.coref_resolved)
displacy.render(doc_resolved_reparsed, style='dep',jupyter=True)

In [13]:
def doc_resolve_render(doc, nlp_with_coref, nlp):
    print("ORIG DOC:",doc)
    doc_resolved = nlp_with_coref(doc)
    print("RESOVED DOC:",doc_resolved._.coref_resolved)
    doc_resolved_reparsed = nlp(doc_resolved._.coref_resolved)
    displacy.render(doc_resolved_reparsed, style='dep',jupyter=True)

In [14]:
doc2 = "The Marzilibahn is 105 meters long. It is the shortest funicular in Europe."
doc_resolve_render(doc2, nlp_with_coref, nlp)

ORIG DOC: The Marzilibahn is 105 meters long. It is the shortest funicular in Europe.
RESOVED DOC: The Marzilibahn is 105 meters long. The Marzilibahn is the shortest funicular in Europe.


In [15]:
doc3 = '''Bern is the capital city of Switzerland. It is also the capital of the canton of Bern. As of early 2006, 127.000 people live in the city'''
doc_resolve_render(doc3,nlp_with_coref, nlp)

ORIG DOC: Bern is the capital city of Switzerland. It is also the capital of the canton of Bern. As of early 2006, 127.000 people live in the city
RESOVED DOC: Bern is the capital city of Switzerland. Bern is also the capital of the canton of Bern. As of early 2006, 127.000 people live in the capital city of Switzerland


In [16]:
doc4 = '''The other university is the University of Zürich. It is the largest university in Switzerland.'''
doc_resolve_render(doc4,nlp_with_coref, nlp)

ORIG DOC: The other university is the University of Zürich. It is the largest university in Switzerland.
RESOVED DOC: The other university is the University of Zürich. The other university is the University of Zürich.


## Adding coreference information to the JSON representation
Unfortunately, we cannot directly inform the to_json() method of spacy to add the coreference resolution information to the parse. However, we can define a function that returns a JSON representation with additional information on the token level that can be used for information extraction.

Therefore we introduce 3 additional attributes on the token level of spacy:
  - coref_cluster_id
  - main_coref_start: First token that is part of this coreferential expression
  - main_coref_end: Index of token coming after the last part of this coreferential expression
  
 main_coref_start and main_coref_end used the normal Spacy span conventions to describe a slice of tokens in a document: doc[main_coref_start:main_coref_end] return the span of tokens belonging to the corerential expression.

In [6]:
def doc_to_json_with_coref(doc):
    """Serialize a spacy document into JSON, enriched by coreference infos at token level"""
    
    json_doc = doc.to_json()
    json_tokens = json_doc["tokens"]
    for i,t in enumerate(doc):
        if t._.in_coref:
            for c in t._.coref_clusters:
                json_tokens[i]["coref_cluster_id"] = c.i
                json_tokens[i]["main_coref_start"] = c.main.start
                json_tokens[i]["main_coref_end"] = c.main.end
    return json_doc

In [7]:
doc_resolved = nlp_with_coref('My sister has a dog. She loves him. But he does not love her.')
doc_to_json_with_coref(doc_resolved)

{'text': 'My sister has a dog. She loves him. But he does not love her.',
 'ents': [],
 'sents': [{'start': 0, 'end': 20},
  {'start': 21, 'end': 35},
  {'start': 36, 'end': 61}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 2,
   'pos': 'DET',
   'tag': 'PRP$',
   'dep': 'poss',
   'head': 1,
   'coref_cluster_id': 0,
   'main_coref_start': 0,
   'main_coref_end': 2},
  {'id': 1,
   'start': 3,
   'end': 9,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'nsubj',
   'head': 2,
   'coref_cluster_id': 0,
   'main_coref_start': 0,
   'main_coref_end': 2},
  {'id': 2,
   'start': 10,
   'end': 13,
   'pos': 'VERB',
   'tag': 'VBZ',
   'dep': 'ROOT',
   'head': 2},
  {'id': 3,
   'start': 14,
   'end': 15,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 4,
   'coref_cluster_id': 1,
   'main_coref_start': 3,
   'main_coref_end': 5},
  {'id': 4,
   'start': 16,
   'end': 19,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'dobj',
   'head': 2,
   'coref_cluster_id': 1,
   'main_coref_

Accessing coreference attributes: https://github.com/huggingface/neuralcoref
 - Coreferences are sets (=clusters) of coreferential spans in the document

In [17]:
doc = nlp_with_coref('My sister has a dog. She loves him')

# Ex: doc._.coref_clusters[1].mentions[-1].start will give you the index of the first token of the last mention of the second coreference cluster in the document.

In [18]:
doc._.coref_clusters[1].mentions[-1].start

8

In [19]:
doc[8]

him

In [20]:
doc._.coref_clusters[1]

a dog: [a dog, him]

In [22]:
doc._.coref_clusters[1].mentions[0].start, doc._.coref_clusters[1].mentions[0].end

(3, 5)

In [23]:
doc[3:5]

a dog